In [2]:
import pandas as pd
import numpy as np
concrete_data = pd.read_csv('https://raw.githubusercontent.com/hsgeral/IBM-AI-Engineering-Professional-Certificate/main/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

## Split data into predictors and target¶

In [4]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [5]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [6]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [7]:
n_cols = predictors.shape[1] # number of predictors

## Start the Neural Network part

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Defining a function that creates a NN with 
- One hidden layer of num_nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error  as the loss function.

In [9]:
# define regression model
def regression_model_1hidden(num_nodes, num_inputs, num_outputs):
    # create model
    model = Sequential()
    model.add(Dense(num_nodes, activation='relu', input_shape=(num_inputs,)))
    model.add(Dense(num_outputs))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and Test the Network
- using the fit method to train and test the model at the same time,
- leaving out 30% of the data for validation 
- using 50 epochs to train the model.
- repeating 50 times to collect the 50 mean squared errors.

The model will set apart the 'validation_split' fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 

In [10]:
num_nodes = 10
num_inputs = n_cols
num_outputs = 1
mse_list = [] # Initialize an empty list to store mean squared errors
for i in range(50):
    model = regression_model_1hidden(num_nodes, num_inputs, num_outputs) # build the model
    history=model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=0,shuffle=True) # fit the model
    mse_list.append(history.history['loss'][-1]) # get the mean square error after the 50th epoch
    print(i,mse_list[i])
# in each of the 50 repetitions the model was recreated otherwise the loss would constinously decrease as we were 
# fitting 50 * 50 epochs, increasing the number of epochs would result in much lower losses, but the exercised required to 
# limit it to 50
          
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print (mean_mse,std_mse)

2024-06-20 17:34:42.289332: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-06-20 17:34:42.294389: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394310000 Hz
2024-06-20 17:34:42.294950: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560501b846e0 executing computations on platform Host. Devices:
2024-06-20 17:34:42.294998: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-06-20 17:34:42.364718: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

0 224.90335993032681
1 125.48393737327375
2 98.0621093982797
3 279.9828900087227
4 302.95782936297246
5 275.86264588333535
6 597.7943825477039
7 135.20196067609007
8 154.9924790465716
9 104.203389298734
10 510.6841392569998
11 218.07988748537187
12 477.51319366751363
13 382.65455559718623
14 1007.9880868856189
15 109.48029204115953
16 110.97963089222056
17 235.3313394194668
18 143.67277878167394
19 889.5215774938237
20 120.37807661418941
21 283.67108243183026
22 199.9362551071442
23 406.887052073062
24 294.2594483860984
25 1063.4853597738872
26 349.0095390499713
27 661.08147575389
28 125.92287159362877
29 671.945507033688
30 157.88533750155761
31 322.68059490061006
32 156.72989742368998
33 170.5738365607189
34 796.7643828795456
35 296.07854989695977
36 240.1128306183835
37 615.4291573152793
38 276.79544334041265
39 137.71661269019944
40 144.2256111430725
41 224.83486182944287
42 106.89304275618512
43 363.86142898961674
44 142.15146741721568
45 2772.414327296073
46 589.0904148223497
47 